In [32]:
from datetime import date, timedelta, datetime
import numpy as np
import pandas as pd
from pathlib import Path
import sqlite3
import warnings
import sys
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint


sys.path.append('C:/Users/Jordan Nishimura/NBA_Model_v1')

from src.data.process_data_no_split import get_data_from_db_all



In [26]:
start_season = 2013
end_season = 2022
path_to_db = Path.home().joinpath('NBA_model_v1', 'data', 'nba.db')
X, y, df = get_data_from_db_all(target = ['HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE'], db_filepath = path_to_db, table = 'team_stats_ewa_matchup_prod')


C:\Users\Jordan Nishimura\NBA_model_v1\data\nba.db


In [27]:
X

,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_AST_L5,HOME_STL_L5,...,AWAY_EFG_PCT_L20,AWAY_EFG_PCT_opp_L20,AWAY_AST_RATIO_L20,AWAY_AST_RATIO_opp_L20,AWAY_TOV_PCT_L20,AWAY_TOV_PCT_opp_L20,AWAY_PIE_L20,AWAY_REST,AWAY_Threein4B2B,AWAY_Threein4
0,31.518241,66.650047,11.148672,22.113305,21.954930,25.266237,14.512977,33.514702,23.799377,6.896403,...,0.514710,0.500641,24.305035,23.174242,12.574422,14.664175,0.503846,3.0,0,0
1,29.772751,66.803971,8.735208,19.678742,19.213618,27.270855,15.839389,36.529798,20.015921,9.338738,...,0.545240,0.483274,24.820241,21.489581,15.404898,17.281397,0.555548,1.0,1,0
2,34.273504,69.759524,3.891372,17.806949,20.125979,27.945182,11.645941,38.028180,20.752197,7.152998,...,0.514862,0.476606,23.658670,20.646725,14.860777,13.379923,0.514737,2.0,0,1
3,33.390751,69.422949,6.394982,23.334510,13.025644,19.490081,12.977680,33.674126,21.252429,6.894249,...,0.498716,0.472069,26.011999,25.555622,12.926552,13.817702,0.530071,2.0,0,0
4,34.559555,75.184094,7.061383,17.070850,19.556235,24.876820,13.514408,32.649867,21.339324,8.288540,...,0.523593,0.489991,24.703460,21.055164,13.115250,12.220492,0.544858,2.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12247,27.166813,47.628239,13.123677,41.327325,21.489388,26.825056,12.269685,34.412481,23.934846,7.311096,...,0.539847,0.539785,26.038283,25.527420,11.262276,12.645388,0.506296,2.0,0,0
12248,32.151412,55.700324,13.890367,35.667494,18.796397,22.687808,10.305881,34.151402,28.346439,5.963433,...,0.540340,0.530662,26.443437,25.109101,11.484703,12.828612,0.514665,3.0,0,0
12249,32.224255,55.771542,12.127628,33.455749,18.265699,22.293664,9.021125,36.746096,29.291891,5.409314,...,0.529404,0.533098,26.782842,25.899091,11.117936,12.596284,0.502954,3.0,0,0
12250,27.422262,58.879823,14.464445,37.135502,14.758088,17.708631,10.687332,29.475882,25.943658,6.682548,...,0.553990,0.520261,29.353385,26.822104,11.303343,9.768286,0.544986,2.0,0,0


In [28]:
def season_to_string(x):
    return str(x) + '-' + str(x+1)[2:]

In [29]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=3)
tscv.split()

TypeError: split() missing 1 required positional argument: 'X'

In [46]:
# Define the season indices (replace with your actual season indices)
season_indices = []
for season in range(2013, 2023):
    season_string = season_to_string(season)
    indices = df.loc[df['SEASON'] == season_string].index
    season_indices.append(indices)
    
# Set the number of threads for parallel execution
num_threads = 4  # Adjust the number of threads as needed

tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)

# # Enable multi-threading for TensorFlow
# with tf.compat.v2.Session(config=tf.compat.v2.ConfigProto(
#     intra_op_parallelism_threads=num_threads,
#     inter_op_parallelism_threads=num_threads,
# )) as sess:
        
    # Perform custom time series cross-validation
    
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')
    
season = 2017

validation_scores = []
score_predictions = []

for i in range(4, len(season_indices)):
# for i in range(4, 5):
    
    season_string = season_to_string(season)
    
    # Get the outer training and testing indices based on the season indices
    outer_train_indices = np.concatenate(season_indices[0:i])
    outer_test_indices = season_indices[i]

    # Split the data into outer training and testing sets based on the indices
    outer_train_X, outer_test_X = X.loc[outer_train_indices], X.loc[outer_test_indices]
    outer_train_y, outer_test_y = y.loc[outer_train_indices], y.loc[outer_test_indices]

    # Normalize the outer training and testing features
    X_mean, X_std = np.mean(outer_train_X, axis=0), np.std(outer_train_X, axis=0)
    outer_train_X_normalized = (outer_train_X - X_mean) / X_std
    outer_test_X_normalized = (outer_test_X - X_mean) / X_std

    best_epochs = None
    best_loss = float('inf')

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(X.shape[1],)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(2)
    ])
    
    # Compile the model
    model.compile(optimizer=tf.optimizers.AdamW(), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mean_absolute_error'])


    # Train the final model on the outer training set with the best number of epochs
    model.fit(outer_train_X_normalized, outer_train_y, epochs=50, batch_size=32
              ,verbose=0, use_multiprocessing=True, workers=2, validation_split=0.1, callbacks=[earlyStopping])

    # Evaluate the model on the outer test set
    loss = model.evaluate(outer_test_X_normalized, outer_test_y)
    print(f"Test loss/rmse/mae for season {season_string}: {loss}")
    validation_scores.append(loss)

    # Make predictions
    predictions = model.predict(outer_test_X_normalized)
    
    score_predictions.append(predictions)


41/41 [==============================] - 0s 2ms/step - loss: 435.9876 - root_mean_squared_error: 20.8803 - mean_absolute_error: 16.8168
Test loss/rmse/mae for season 2017-18: [435.98760986328125, 20.88031578063965, 16.816762924194336]
41/41 [==============================] - 0s 2ms/step - loss: 621.4402 - root_mean_squared_error: 24.9287 - mean_absolute_error: 20.9941
Test loss/rmse/mae for season 2017-18: [621.440185546875, 24.928701400756836, 20.994075775146484]
35/35 [==============================] - 0s 2ms/step - loss: 309.5329 - root_mean_squared_error: 17.5935 - mean_absolute_error: 14.0699
Test loss/rmse/mae for season 2017-18: [309.5329284667969, 17.593547821044922, 14.069925308227539]
36/36 [==============================] - 0s 2ms/step - loss: 212.5375 - root_mean_squared_error: 14.5787 - mean_absolute_error: 11.6456
Test loss/rmse/mae for season 2017-18: [212.53750610351562, 14.578665733337402, 11.645559310913086]
39/39 [==============================] - 0s 2ms/step - loss:

In [47]:
print(validation_scores)
print(np.array(validation_scores).mean())

[[435.98760986328125, 20.88031578063965, 16.816762924194336], [621.440185546875, 24.928701400756836, 20.994075775146484], [309.5329284667969, 17.593547821044922, 14.069925308227539], [212.53750610351562, 14.578665733337402, 11.645559310913086], [186.33470153808594, 13.650447845458984, 10.900707244873047], [205.838623046875, 14.347077369689941, 11.337647438049316]]
120.18972158432007


In [21]:
start_season = 2013
end_season = 2022
path_to_db = Path.home().joinpath('NBA_model_v1', 'data', 'nba.db')
X, y_win, df = get_data_from_db_all(target = 'HOME_WL', db_filepath = path_to_db, table = 'team_stats_ewa_matchup_prod')


C:\Users\Jordan Nishimura\NBA_model_v1\data\nba.db


In [48]:
# Define the season indices (replace with your actual season indices)
season_indices = []
for season in range(2013, 2023):
    season_string = season_to_string(season)
    indices = df.loc[df['SEASON'] == season_string].index
    season_indices.append(indices)
    
# Set the number of threads for parallel execution
num_threads = 4  # Adjust the number of threads as needed

# tf.config.threading.set_inter_op_parallelism_threads(num_threads)
# tf.config.threading.set_intra_op_parallelism_threads(num_threads)

# # Enable multi-threading for TensorFlow
# with tf.compat.v2.Session(config=tf.compat.v2.ConfigProto(
#     intra_op_parallelism_threads=num_threads,
#     inter_op_parallelism_threads=num_threads,
# )) as sess:
        
    # Perform custom time series cross-validation
season = 2017

earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')


validation_scores_win_pred = []
score_predictions_win_pred = []
epoch_list_win_pred = []

for i in range(4, len(season_indices)):
# for i in range(4, 5):
    
    season_string = season_to_string(season)
    
    # Get the outer training and testing indices based on the season indices
    outer_train_indices = np.concatenate(season_indices[0:i])
    outer_test_indices = season_indices[i]

    # Split the data into outer training and testing sets based on the indices
    outer_train_X, outer_test_X = X.loc[outer_train_indices], X.loc[outer_test_indices]
    outer_train_y, outer_test_y = y_win.loc[outer_train_indices], y_win.loc[outer_test_indices]

    # Normalize the outer training and testing features
    X_mean, X_std = np.mean(outer_train_X, axis=0), np.std(outer_train_X, axis=0)
    outer_train_X_normalized = (outer_train_X - X_mean) / X_std
    outer_test_X_normalized = (outer_test_X - X_mean) / X_std

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(X.shape[1],)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    
    # Compile the model
    model.compile(optimizer=tf.optimizers.AdamW(), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the final model on the outer training set with the best number of epochs
    model.fit(outer_train_X_normalized, outer_train_y, epochs=50, batch_size=32
              ,verbose=0, use_multiprocessing=True, workers=2, validation_split=0.1, callbacks=[earlyStopping])

    # Evaluate the model on the outer test set
    loss = model.evaluate(outer_test_X_normalized, outer_test_y)
    print(f"Test loss/accuracy for season {season_string}: {loss}")
    validation_scores_win_pred.append(loss)

    # Make predictions
    predictions = model.predict(outer_test_X_normalized)
    
    score_predictions_win_pred.append(predictions)
    season+=1

c:\Users\Jordan Nishimura\NBA_Model_v1\nba-model-venv\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


41/41 [==============================] - 0s 2ms/step - loss: 9.0455 - accuracy: 0.4136
Test loss/accuracy for season 2017-18: [9.04549503326416, 0.4135802388191223]
41/41 [==============================] - 0s 2ms/step


c:\Users\Jordan Nishimura\NBA_Model_v1\nba-model-venv\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


41/41 [==============================] - 0s 2ms/step - loss: 6.2646 - accuracy: 0.5892
Test loss/accuracy for season 2018-19: [6.264552116394043, 0.5891891717910767]
41/41 [==============================] - 0s 2ms/step


c:\Users\Jordan Nishimura\NBA_Model_v1\nba-model-venv\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


...

36/36 [==============================] - 0s 2ms/step - loss: 6.6998 - accuracy: 0.5640
Test loss/accuracy for season 2020-21: [6.699845314025879, 0.5639686584472656]
36/36 [==============================] - 0s 2ms/step


c:\Users\Jordan Nishimura\NBA_Model_v1\nba-model-venv\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


39/39 [==============================] - 0s 2ms/step - loss: 6.8955 - accuracy: 0.5478
Test loss/accuracy for season 2021-22: [6.8955230712890625, 0.5478119850158691]
39/39 [==============================] - 0s 2ms/step


c:\Users\Jordan Nishimura\NBA_Model_v1\nba-model-venv\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


41/41 [==============================] - 0s 2ms/step - loss: 6.4195 - accuracy: 0.5790
Test loss/accuracy for season 2022-23: [6.419494152069092, 0.5790285468101501]
41/41 [==============================] - 0s 2ms/step


In [50]:
validation_scores_win_pred

[[9.04549503326416, 0.4135802388191223],
 [6.264552116394043, 0.5891891717910767],
 [6.912805557250977, 0.5466786623001099],
 [6.699845314025879, 0.5639686584472656],
 [6.8955230712890625, 0.5478119850158691],
 [6.419494152069092, 0.5790285468101501]]